# Phase 6: Integrated Clinical Decision Support (CDS)

This notebook integrates Phase 3–5 models to generate real-time recommendations, alerts, and risk scores suitable for clinician-facing dashboards and bedside devices.

## 1. Import Models & Utilities

In [ ]:
import os
import json
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

# Load trained models
with open(os.path.join('..','models','temperature_optimization_model.pkl'),'rb') as f:
    temp_model = pickle.load(f)
with open(os.path.join('..','models','temperature_model_features.json'),'r') as f:
    temp_features = json.load(f)

# Phase 4 models (load best variants if present)
def safe_load_model(path):
    try:
        with open(path,'rb') as f:
            return pickle.load(f)
    except Exception:
        return None

seizure_model = safe_load_model(os.path.join('..','models','seizure_model_logreg.pkl')) or \
                safe_load_model(os.path.join('..','models','seizure_model_rf.pkl')) or \
                safe_load_model(os.path.join('..','models','seizure_model_gb.pkl'))
sepsis_model  = safe_load_model(os.path.join('..','models','sepsis_model_rf.pkl'))
cardiac_model = safe_load_model(os.path.join('..','models','cardiac_model_rf.pkl'))
renal_model   = safe_load_model(os.path.join('..','models','renal_model_rf.pkl'))

# Phase 5 prognosis model
with open(os.path.join('..','models','prognosis_model_logreg.pkl'),'rb') as f:
    prognosis_model = pickle.load(f)
with open(os.path.join('..','models','prognosis_feature_columns.json'),'r') as f:
    prognosis_features = json.load(f)

print("Models loaded. Ready for CDS inference.")

In [ ]:
# Configuration: thresholds and mappings
CDS_CONFIG = {
    'seizure': {'high': 0.70, 'medium': 0.40},
    'sepsis':  {'high': 0.65, 'medium': 0.35},
    'cardiac': {'high': 0.60, 'medium': 0.30},
    'renal':   {'high': 0.60, 'medium': 0.30},
    'prognosis': {'poor_outcome_prob_high': 0.65, 'poor_outcome_prob_medium': 0.40},
    'temperature_adjustment_degC': {'max': 1.0, 'medium': 0.5}
}

RISK_LABELS = { 'high': 'HIGH', 'medium': 'MED', 'low': 'LOW' }

# Helper: categorize probability by thresholds
def categorize(prob: float, high: float, medium: float):
    if prob >= high:
        return RISK_LABELS['high']
    if prob >= medium:
        return RISK_LABELS['medium']
    return RISK_LABELS['low']

# Helper: compute recommendation text snippets
def build_recommendations(risks: dict, temp_delta: float):
    recs = []
    if risks.get('seizure') == 'HIGH':
        recs.append('Initiate continuous EEG; review antiseizure therapy.')
    elif risks.get('seizure') == 'MED':
        recs.append('Increase neuro checks frequency; consider EEG if symptomatic.')

    if risks.get('sepsis') == 'HIGH':
        recs.append('Sepsis bundle: cultures, antibiotics, fluids per protocol.')
    elif risks.get('sepsis') == 'MED':
        recs.append('Trend lactate; monitor vitals and labs closely.')

    if risks.get('cardiac') == 'HIGH':
        recs.append('Cardiac consult; optimize MAP and rhythm management.')
    elif risks.get('cardiac') == 'MED':
        recs.append('Increase telemetry vigilance; review medications impacting QT/MAP.')

    if risks.get('renal') == 'HIGH':
        recs.append('Renal consult; adjust nephrotoxic meds; optimize fluids.')
    elif risks.get('renal') == 'MED':
        recs.append('Monitor urine output and creatinine; adjust dosing.')

    if risks.get('prognosis') == 'HIGH':
        recs.append('Discuss goals of care; consider advanced monitoring/support.')
    elif risks.get('prognosis') == 'MED':
        recs.append('Ensure multidisciplinary review; reassess trajectory in 12h.')

    if temp_delta >= CDS_CONFIG['temperature_adjustment_degC']['max']:
        recs.append('Strongly consider temperature reduction by ~1.0°C.')
    elif temp_delta >= CDS_CONFIG['temperature_adjustment_degC']['medium']:
        recs.append('Consider temperature reduction by ~0.5°C.')
    else:
        recs.append('Maintain current temperature; continue monitoring.')

    return recs

print('CDS config and helpers ready.')

In [ ]:
# Inference utilities: compute risks and scorecard from a single row of features

def compute_temp_adjustment(X_temp_row: pd.Series):
    # Predict temperature delta (positive means reduce temp)
    try:
        delta = float(temp_model.predict(X_temp_row[temp_features].values.reshape(1,-1))[0])
    except Exception:
        delta = 0.0
    return max(0.0, delta)


def compute_event_probability(model, X_row: pd.Series, cols: list):
    if model is None:
        return None
    try:
        prob = float(model.predict_proba(X_row[cols].values.reshape(1,-1))[0,1])
        return prob
    except Exception:
        return None


def build_scorecard(patient_id: str, X_row: pd.Series):
    # Temperature delta from Phase 3
    temp_delta = compute_temp_adjustment(X_row)

    # Phase 4 risks
    seizure_prob = compute_event_probability(seizure_model, X_row, prognosis_features)  # fallback to prognosis_features if phase4 list unavailable
    sepsis_prob  = compute_event_probability(sepsis_model,  X_row, prognosis_features)
    cardiac_prob = compute_event_probability(cardiac_model, X_row, prognosis_features)
    renal_prob   = compute_event_probability(renal_model,   X_row, prognosis_features)

    # Phase 5 prognosis
    prognosis_prob = compute_event_probability(prognosis_model, X_row, prognosis_features)

    risks = {
        'seizure': categorize(seizure_prob or 0.0, CDS_CONFIG['seizure']['high'], CDS_CONFIG['seizure']['medium']),
        'sepsis':  categorize(sepsis_prob or 0.0,  CDS_CONFIG['sepsis']['high'],  CDS_CONFIG['sepsis']['medium']),
        'cardiac': categorize(cardiac_prob or 0.0, CDS_CONFIG['cardiac']['high'], CDS_CONFIG['cardiac']['medium']),
        'renal':   categorize(renal_prob or 0.0,   CDS_CONFIG['renal']['high'],   CDS_CONFIG['renal']['medium']),
        'prognosis': categorize(prognosis_prob or 0.0, CDS_CONFIG['prognosis']['poor_outcome_prob_high'], CDS_CONFIG['prognosis']['poor_outcome_prob_medium'])
    }

    recs = build_recommendations(risks, temp_delta)

    scorecard = {
        'patient_id': patient_id,
        'timestamp': datetime.utcnow().isoformat() + 'Z',
        'probabilities': {
            'seizure': seizure_prob,
            'sepsis': sepsis_prob,
            'cardiac': cardiac_prob,
            'renal': renal_prob,
            'prognosis_poor_outcome': prognosis_prob
        },
        'risk_levels': risks,
        'temperature_adjustment_degC': round(temp_delta, 2),
        'recommendations': recs
    }
    return scorecard

print('Inference utilities ready.')

In [ ]:
# Batch processing: build scorecards for a dataframe of patients

def run_cds(df: pd.DataFrame, id_col: str = 'patient_id'):
    scorecards = []
    for _, row in df.iterrows():
        pid = str(row[id_col]) if id_col in df.columns else 'unknown'
        scorecards.append(build_scorecard(pid, row))
    return scorecards

# Persistence: save outputs for downstream APIs

def save_cds_outputs(scorecards, out_dir='../outputs/cds'):
    os.makedirs(out_dir, exist_ok=True)
    ts = datetime.utcnow().strftime('%Y%m%dT%H%M%SZ')
    out_path = os.path.join(out_dir, f'cds_scorecards_{ts}.json')
    with open(out_path, 'w') as f:
        json.dump({'generated_at': ts, 'items': scorecards}, f, indent=2)
    print(f'Saved {len(scorecards)} scorecards to {out_path}')
    return out_path

print('Batch + persistence utilities ready.')

In [ ]:
# Demo: create a small synthetic input if no dataset is provided

# Try loading from a canonical features CSV if present
candidate_paths = [
    os.path.join('..','data','phase5_features.csv'),
    os.path.join('..','data','phase4_features.csv')
]
loaded_df = None
for p in candidate_paths:
    if os.path.exists(p):
        try:
            loaded_df = pd.read_csv(p)
            print(f'Loaded dataset from {p} with shape {loaded_df.shape}')
            break
        except Exception as e:
            print(f'Failed to load {p}: {e}')

if loaded_df is None:
    np.random.seed(42)
    # Minimal set using prognosis_features as columns
    cols = prognosis_features if isinstance(prognosis_features, list) else []
    if not cols:
        cols = ['feat1','feat2','feat3']
    demo_data = {c: np.random.normal(0,1, size=5) for c in cols}
    demo_data['patient_id'] = [f'DEMO-{i+1:03d}' for i in range(5)]
    loaded_df = pd.DataFrame(demo_data)
    print('Generated synthetic demo dataframe:', loaded_df.shape)

scorecards = run_cds(loaded_df, id_col='patient_id')
out_path = save_cds_outputs(scorecards)

# Preview first two scorecards
print(json.dumps(scorecards[:2], indent=2))

In [ ]:
# Visualization: simple risk barplot and alert badges for one patient

if len(scorecards) > 0:
    sc = scorecards[0]
    probs = sc['probabilities']
    labels = sc['risk_levels']

    plt.figure(figsize=(8,4))
    keys = ['seizure','sepsis','cardiac','renal','prognosis_poor_outcome']
    vals = [probs[k] if probs[k] is not None else 0.0 for k in keys]
    sns.barplot(x=keys, y=vals, palette='Reds')
    plt.title(f"Risk probabilities for {sc['patient_id']}\nAdj temp: {sc['temperature_adjustment_degC']}°C")
    plt.ylim(0,1)
    plt.xticks(rotation=30)
    plt.show()

    print('Risk levels:', labels)
    print('Recommendations:')
    for r in sc['recommendations']:
        print('-', r)
else:
    print('No scorecards to visualize.')